In [24]:
import numpy as np
import pandas as pd
import json
import math

In [44]:
# root = '../src/existing_models/contextualizing-hate-speech-models-with-explanations-master/runs/'
# models = ['twitter_es_reg_nb5_h5_is_bal_pos_seed_',]
# seeds = np.arange(10).tolist()
# tasks = ['gab', 'ws', 'twitter', 'nyt']
#
# result = None
# for model in models:
#     seed = 9 # for seed in seeds:
#     task = 'twitter' #for task in tasks:
#     f = pd.read_csv(f'{root}{model}{seed}/eval_results_0_test_{task}.txt', delimiter='=', header=None)
#     f = f.append([['task', task]], ignore_index=True)
#     f = f.append([['model', model]], ignore_index=True)
#     f = f.append([['seed', seed]], ignore_index=True)
#     f.set_index(0, inplace=True)
#     f = f.transpose()
#     if result is None:
#         result = f
#     else:
#         result = pd.concat([result, f], ignore_index=True, sort=False)
#

In [45]:
# result.to_csv(f'{root}/results_agg_11_01_2020.csv')

,acc,auc_roc,disparate_impact_0.6,disparate_impact_0.8,eval_loss,eval_loss_reg,f1,global_step,loss,precision,...,priv_ratio_0.6,priv_ratio_0.8,recall,unpriv_n_0.6,unpriv_n_0.8,unpriv_ratio_0.6,unpriv_ratio_0.8,task,model,seed
1,0.7705436762872758,0.8689629283877215,0.8203244389381303,0.8624290326171657,0.4404911465165805,0.0,0.4352135945244277,0,None,0.2960822093770071,...,0.8457142857142858,0.8125,0.8210151380231523,9904,10365,0.6937600969305331,0.7007235890014472,twitter,twitter_es_reg_nb5_h5_is_bal_pos_seed_,9


In [43]:
# result = pd.concat([f, f], ignore_index=True, sort=False) #todo deal with missing columns
# result

,acc,auc_roc,disparate_impact_0.6,disparate_impact_0.8,eval_loss,eval_loss_reg,f1,global_step,loss,precision,...,priv_ratio_0.6,priv_ratio_0.8,recall,unpriv_n_0.6,unpriv_n_0.8,unpriv_ratio_0.6,unpriv_ratio_0.8,task,model,seed
0,0.7705436762872758,0.8689629283877215,0.8203244389381303,0.8624290326171657,0.4404911465165805,0.0,0.4352135945244277,0,None,0.2960822093770071,...,0.8457142857142858,0.8125,0.8210151380231523,9904,10365,0.6937600969305331,0.7007235890014472,twitter,twitter_es_reg_nb5_h5_is_bal_pos_seed_,9
1,0.7705436762872758,0.8689629283877215,0.8203244389381303,0.8624290326171657,0.4404911465165805,0.0,0.4352135945244277,0,None,0.2960822093770071,...,0.8457142857142858,0.8125,0.8210151380231523,9904,10365,0.6937600969305331,0.7007235890014472,twitter,twitter_es_reg_nb5_h5_is_bal_pos_seed_,9


In [25]:
results = pd.read_csv('../data/contextualize_results/results_agg_11_23_2020.csv', index_col=0)
results['model'].unique()
# results['task'].unique()

array(['gab', 'ws', 'twitter', 'nyt', 'twitter_harass'], dtype=object)

In [26]:
for model in results.model.unique():
    for task in results.task.unique():
        temp_df_all_seeds = results[(results['model']==model) & (results['task']==task)]
        temp_df_all_seeds.to_csv(f'../data/contextualize_results/all_seeds/{model}{task}.csv')

In [27]:
results.columns

Index(['acc ', 'auc_roc ', 'disparate_impact_0.6 ', 'disparate_impact_0.8 ',
       'eval_loss ', 'eval_loss_reg ', 'f1 ', 'f1_weighted ', 'global_step ',
       'loss ', 'precision ', 'precision_weighted ', 'priv_n_0.6 ',
       'priv_n_0.8 ', 'priv_ratio_0.6 ', 'priv_ratio_0.8 ', 'recall ',
       'recall_weighted ', 'unpriv_n_0.6 ', 'unpriv_n_0.8 ',
       'unpriv_ratio_0.6 ', 'unpriv_ratio_0.8 ', 'task', 'model', 'seed'],
      dtype='object')

In [28]:
def ci_sample(series):
    pop_mean = series.mean()
    pop_std = series.std(ddof=0)
    pop_n =  len(series) - 1
    return pop_mean - (1.96*pop_std/math.sqrt(pop_n)), pop_mean + (1.96*pop_std/math.sqrt(pop_n))

In [29]:

results.columns
results['disparate_impact_0.8 '] = results['disparate_impact_0.8 '].astype('float')
results['disparate_impact_0.6 '] = results['disparate_impact_0.6 '].astype('float')

In [30]:
temp = results.groupby(['model', 'task']).agg(
    acc_mean=('acc ', 'mean'),
    acc_std=('acc ', 'std'),
    acc_ci=('acc ', ci_sample),
    auc_roc_mean=('auc_roc ', 'mean'),
    auc_roc_std=('auc_roc ', 'std'),
    auc_roc_ci=('auc_roc ', ci_sample),
    f1_mean=('f1 ', 'mean'),
    f1_std=('f1 ', 'std'),
    f1_ci=('f1 ', ci_sample),
    precision_mean=('precision ', 'mean'),
    precision_std=('precision ', 'std'),
    precision_ci=('precision ', ci_sample),
    recall_mean=('recall ', 'mean'),
    recall_std=('recall ', 'std'),
    recall_ci=('recall ', ci_sample),
    precision_w_mean=('precision_weighted ', 'mean'),
    precision_w_std=('precision_weighted ', 'std'),
    precision_w_ci=('precision_weighted ', ci_sample),
    recall_w_mean=('recall_weighted ', 'mean'),
    recall_w_std=('recall_weighted ', 'std'),
    recall_w_ci=('recall_weighted ', ci_sample),
    f1_w_mean=('f1_weighted ', 'mean'),
    f1_w_std=('f1_weighted ', 'std'),
    f1_w_ci=('f1_weighted ', ci_sample),
    disparate_impact_06_mean=('disparate_impact_0.6 ', np.mean),
    disparate_impact_06_std=('disparate_impact_0.6 ', np.std),
    disparate_impact_06_ci=('disparate_impact_0.6 ',ci_sample),
    disparate_impact_08_mean=('disparate_impact_0.8 ', np.mean),
    disparate_impact_08_std=('disparate_impact_0.8 ',np.std),
    disparate_impact_08_ci=('disparate_impact_0.8 ', ci_sample),
    priv_n_06=('priv_n_0.6 ', np.median),
    priv_n_08=('priv_n_0.8 ', np.median),
    unpriv_n_06=('unpriv_n_0.6 ', np.median),
    unpriv_n_08=('unpriv_n_0.8 ', np.median),
).reset_index().sort_values(by=['model', 'task'])
temp

,model,task,acc_mean,acc_std,acc_ci,auc_roc_mean,auc_roc_std,auc_roc_ci,f1_mean,f1_std,...,disparate_impact_06_mean,disparate_impact_06_std,disparate_impact_06_ci,disparate_impact_08_mean,disparate_impact_08_std,disparate_impact_08_ci,priv_n_06,priv_n_08,unpriv_n_06,unpriv_n_08
0,majority_gab_es_reg_nb0_h1_bal_seed_,gab,0.862108,0.012026,"(0.8546544856593332, 0.8695623818105465)",0.874507,0.007422,"(0.8699068385251156, 0.8791072046781914)",0.449921,0.018215,...,1.581563,0.638837,"(1.1856077090048187, 1.9775184225582472)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
1,majority_gab_es_reg_nb0_h1_bal_seed_,nyt,0.917362,0.024158,"(0.9023890204927413, 0.9323356171884182)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
2,majority_gab_es_reg_nb0_h1_bal_seed_,twitter,0.799079,0.017829,"(0.7880290003327614, 0.8101299794351934)",0.739081,0.009118,"(0.7334297289437248, 0.7447327526196015)",0.310914,0.016020,...,1.757463,0.192340,"(1.6382492846163017, 1.8766758280603493)",2.507590,0.451256,"(2.2278989080454092, 2.7872813465869815)",525.0,64.0,9904.0,10365.0
3,majority_gab_es_reg_nb0_h1_bal_seed_,ws,0.872146,0.009383,"(0.8663306797820927, 0.8779615576608297)",0.828538,0.010412,"(0.822084203132161, 0.8349908206095297)",0.462081,0.043819,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
4,majority_gab_es_reg_nb5_h5_is_bal_pos_seed_,gab,0.881928,0.020701,"(0.8690969473006679, 0.8947584743860789)",0.886380,0.010175,"(0.8800729619505216, 0.8926863453965002)",0.480698,0.030686,...,1.286723,0.038792,"(1.2626797696530172, 1.3107662170699763)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
5,majority_gab_es_reg_nb5_h5_is_bal_pos_seed_,nyt,0.872833,0.039768,"(0.848184690859489, 0.8974819758071777)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
6,majority_gab_es_reg_nb5_h5_is_bal_pos_seed_,twitter,0.827117,0.020782,"(0.8142356392423945, 0.8399977484266054)",0.742117,0.008340,"(0.7369475537143811, 0.7472862318210093)",0.338319,0.016135,...,1.484461,0.201475,"(1.359585089673893, 1.6093359280251855)",2.070144,0.510267,"(1.753876940510465, 2.3864110749132044)",525.0,64.0,9904.0,10365.0
7,majority_gab_es_reg_nb5_h5_is_bal_pos_seed_,ws,0.866667,0.012307,"(0.8590387528229122, 0.874294580510421)",0.831201,0.010391,"(0.8247609941413466, 0.8376416649279793)",0.456342,0.021947,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
8,majority_gab_es_vanilla_bal_seed_,gab,0.882831,0.015758,"(0.8730642575235151, 0.8925983930788944)",0.891738,0.007496,"(0.8870917266629993, 0.8963837414831942)",0.488232,0.021547,...,1.582046,0.701131,"(1.1474801485394046, 2.0166115835064615)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
9,majority_gab_es_vanilla_bal_seed_,nyt,0.783710,0.082726,"(0.7324361957562131, 0.8349840940988593)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN


In [31]:
results

,acc,auc_roc,disparate_impact_0.6,disparate_impact_0.8,eval_loss,eval_loss_reg,f1,f1_weighted,global_step,loss,...,priv_ratio_0.8,recall,recall_weighted,unpriv_n_0.6,unpriv_n_0.8,unpriv_ratio_0.6,unpriv_ratio_0.8,task,model,seed
0,0.703614,0.825777,0.980386,inf,0.524668,0.0,0.310924,0.769314,0,None,...,0.000000,0.798561,0.703614,1657.0,1659.0,0.653591,0.654008,gab,twitter_es_reg_nb5_h5_is_bal_pos_seed_,0
1,0.747336,0.731909,NaN,NaN,0.473567,0.0,0.308333,0.786577,0,None,...,NaN,0.513889,0.747336,NaN,NaN,NaN,NaN,ws,twitter_es_reg_nb5_h5_is_bal_pos_seed_,0
2,0.793940,0.871804,0.789072,0.767273,0.414924,0.0,0.450524,0.827680,0,None,...,0.953125,0.784506,0.793940,9904.0,10365.0,0.722940,0.731307,twitter,twitter_es_reg_nb5_h5_is_bal_pos_seed_,0
3,0.833116,0.000000,NaN,NaN,0.348365,0.0,0.000000,0.908962,0,None,...,NaN,0.000000,0.833116,NaN,NaN,NaN,NaN,nyt,twitter_es_reg_nb5_h5_is_bal_pos_seed_,0
4,0.646386,0.820480,0.589620,0.590115,0.621273,0.0,0.283272,0.724932,0,None,...,1.000000,0.834532,0.646386,1657.0,1659.0,0.589620,0.590115,gab,twitter_es_reg_nb5_h5_is_bal_pos_seed_,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,0.593373,0.823346,1.558841,inf,0.868262,0.0,0.279616,0.680141,0,None,...,0.000000,0.942446,0.593373,1657.0,1659.0,0.519614,0.519590,gab,twitter_harass_es_vanilla_bal_seed_,9
506,0.605784,0.771201,NaN,NaN,0.670159,0.0,0.305630,0.678829,0,None,...,NaN,0.791667,0.605784,NaN,NaN,NaN,NaN,ws,twitter_harass_es_vanilla_bal_seed_,9
507,0.510883,0.667810,46.011712,inf,1.715202,0.0,0.292216,0.590350,0,None,...,0.000000,0.937667,0.510883,9904.0,10365.0,0.438207,0.419199,twitter,twitter_harass_es_vanilla_bal_seed_,9
508,0.458043,0.000000,NaN,NaN,0.740530,0.0,0.000000,0.628299,0,None,...,NaN,0.000000,0.458043,NaN,NaN,NaN,NaN,nyt,twitter_harass_es_vanilla_bal_seed_,9


In [38]:
temp.to_csv('../data/contextualize_results/stats_on_tasks.csv', index=False)

In [ ]:
models_dict = {'twitter': {'twitter_es_reg_nb5_h5_is_bal_pos_seed_', 'twitter_es_reg_nb0_h1_bal_seed_', 'twitter_es_vanilla_bal_seed_'},
                'twitter_harass': {'twitter_harass_es_reg_nb5_h5_is_bal_pos_seed_','twitter_harass_es_reg_nb0_h1_bal_seed_','twitter_harass_es_vanilla_bal_seed_'},
               'gab': {'majority_gab_es_vanilla_bal_seed_', 'majority_gab_es_reg_nb0_h1_bal_seed_', 'majority_gab_es_reg_nb5_h5_is_bal_pos_seed_'},
               'ws': {'ws_es_vanilla_bal_seed_', 'ws_es_reg_nb0_h1_bal_seed_','ws_es_reg_nb5_h5_is_bal_pos_seed_'},
               'nyt': {},
               }

In [33]:
task_to_learner_results = None
for task in models_dict.keys():
    if task_to_learner_results is None:
        task_to_learner_results = temp[(temp['model'].isin(models_dict[task])) & (temp['task'] == task)]
    else:
        task_to_learner_results = pd.concat([task_to_learner_results, temp[(temp['model'].isin(models_dict[task])) & (temp['task'] == task)]])
task_to_learner_results

,model,task,acc_mean,acc_std,acc_ci,auc_roc_mean,auc_roc_std,auc_roc_ci,f1_mean,f1_std,...,disparate_impact_06_mean,disparate_impact_06_std,disparate_impact_06_ci,disparate_impact_08_mean,disparate_impact_08_std,disparate_impact_08_ci,priv_n_06,priv_n_08,unpriv_n_06,unpriv_n_08
14,twitter_es_reg_nb0_h1_bal_seed_,twitter,0.740416,0.013804,"(0.7318604701540944, 0.7489718244091427)",0.851023,0.005828,"(0.8474105131000839, 0.8546345486463854)",0.400923,0.009749,...,0.857148,0.044112,"(0.8298066061381789, 0.884489005007136)",0.962671,0.097553,"(0.902206724586391, 1.023134586881783)",525.0,64.0,9904.0,10365.0
18,twitter_es_reg_nb5_h5_is_bal_pos_seed_,twitter,0.780775,0.020300,"(0.7681924281851983, 0.7933570971767733)",0.861636,0.006482,"(0.8576180444353265, 0.8656529654302911)",0.434504,0.012329,...,0.826937,0.060878,"(0.7892042012454198, 0.8646696652235802)",0.866940,0.114318,"(0.7960850410549574, 0.9377945217594896)",525.0,64.0,9904.0,10365.0
22,twitter_es_vanilla_bal_seed_,twitter,0.797047,0.016549,"(0.786789594562518, 0.8073037988596702)",0.867681,0.012040,"(0.8602187635615669, 0.8751441492098829)",0.452028,0.014638,...,0.840683,0.035621,"(0.8186045992182596, 0.8627610571687874)",0.874632,0.077094,"(0.8268481619460178, 0.9224149791085426)",525.0,64.0,9904.0,10365.0
27,twitter_harass_es_reg_nb0_h1_bal_seed_,twitter_harass,0.699126,0.108810,"(0.6316849387126151, 0.7665673015302938)",0.913960,0.096742,"(0.8539989489968628, 0.9739213422284224)",0.724786,0.059479,...,inf,NaN,"(nan, nan)",inf,NaN,"(nan, nan)",656.0,77.0,12847.0,13426.0
32,twitter_harass_es_reg_nb5_h5_is_bal_pos_seed_,twitter_harass,0.798674,0.010532,"(0.7921467442694817, 0.8052019930481517)",0.951445,0.004280,"(0.9487928326630026, 0.9540978517512333)",0.790119,0.007984,...,41.367705,8.509532,"(36.09344237057658, 46.64196711913195)",inf,NaN,"(nan, nan)",656.0,77.0,12847.0,13426.0
37,twitter_harass_es_vanilla_bal_seed_,twitter_harass,0.802177,0.013758,"(0.7936501224867812, 0.8107044653825811)",0.951767,0.005010,"(0.9486616697289599, 0.9548716952451467)",0.793076,0.010919,...,35.196131,11.226862,"(28.23764979208232, 42.15461160470309)",inf,NaN,"(nan, nan)",656.0,77.0,12847.0,13426.0
0,majority_gab_es_reg_nb0_h1_bal_seed_,gab,0.862108,0.012026,"(0.8546544856593332, 0.8695623818105465)",0.874507,0.007422,"(0.8699068385251156, 0.8791072046781914)",0.449921,0.018215,...,1.581563,0.638837,"(1.1856077090048187, 1.9775184225582472)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
4,majority_gab_es_reg_nb5_h5_is_bal_pos_seed_,gab,0.881928,0.020701,"(0.8690969473006679, 0.8947584743860789)",0.886380,0.010175,"(0.8800729619505216, 0.8926863453965002)",0.480698,0.030686,...,1.286723,0.038792,"(1.2626797696530172, 1.3107662170699763)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
8,majority_gab_es_vanilla_bal_seed_,gab,0.882831,0.015758,"(0.8730642575235151, 0.8925983930788944)",0.891738,0.007496,"(0.8870917266629993, 0.8963837414831942)",0.488232,0.021547,...,1.582046,0.701131,"(1.1474801485394046, 2.0166115835064615)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
42,ws_es_reg_nb0_h1_bal_seed_,ws,0.895282,0.012831,"(0.8873287167761869, 0.9032344491294447)",0.880330,0.009994,"(0.8741355728690714, 0.8865244461242809)",0.547665,0.020910,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN


In [34]:
other_tasks_results = temp[(temp['model'].isin(models_dict['twitter'])) & (temp['task'].isin({'gab', 'ws', 'twitter_harass', 'nyt'}))]
other_tasks_results = pd.concat([other_tasks_results, temp[(temp['model'].isin(models_dict['gab'])) & (temp['task'].isin({'twitter', 'twitter_harass','ws', 'nyt'}))]])
other_tasks_results = pd.concat([other_tasks_results, temp[(temp['model'].isin(models_dict['twitter_harass'])) & (temp['task'].isin({'twitter', 'gab', 'ws', 'nyt'}))]])
other_tasks_results = pd.concat([other_tasks_results, temp[(temp['model'].isin(models_dict['ws'])) & (temp['task'].isin({'twitter', 'twitter_harass','gab', 'nyt'}))]])

In [35]:
other_tasks_results

,model,task,acc_mean,acc_std,acc_ci,auc_roc_mean,auc_roc_std,auc_roc_ci,f1_mean,f1_std,...,disparate_impact_06_mean,disparate_impact_06_std,disparate_impact_06_ci,disparate_impact_08_mean,disparate_impact_08_std,disparate_impact_08_ci,priv_n_06,priv_n_08,unpriv_n_06,unpriv_n_08
12,twitter_es_reg_nb0_h1_bal_seed_,gab,0.628976,0.036343,"(0.6064502124205815, 0.651501594808334)",0.818889,0.007008,"(0.8145454343478087, 0.8232331002209861)",0.276995,0.014425,...,0.856639,0.060978,"(0.8188442563563496, 0.8944327502821539)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
13,twitter_es_reg_nb0_h1_bal_seed_,nyt,0.692862,0.067703,"(0.6508996380141923, 0.7348249996669673)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
15,twitter_es_reg_nb0_h1_bal_seed_,ws,0.742009,0.025167,"(0.7264106936761919, 0.7576075711639908)",0.748519,0.014860,"(0.739308099673421, 0.7577289373636157)",0.333568,0.027191,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
16,twitter_es_reg_nb5_h5_is_bal_pos_seed_,gab,0.697771,0.047310,"(0.6684477917724448, 0.7270943769022539)",0.821897,0.006244,"(0.8180272449425315, 0.8257673052161579)",0.310001,0.022413,...,1.219403,0.461532,"(0.9333420557721496, 1.5054630136303855)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
17,twitter_es_reg_nb5_h5_is_bal_pos_seed_,nyt,0.794993,0.068235,"(0.7527000530041628, 0.8372854542422141)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
19,twitter_es_reg_nb5_h5_is_bal_pos_seed_,ws,0.771081,0.030103,"(0.7524226540514041, 0.7897386853702091)",0.750957,0.017452,"(0.740140144293877, 0.7617734359530367)",0.336322,0.026036,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
20,twitter_es_vanilla_bal_seed_,gab,0.766506,0.026450,"(0.7501121545689884, 0.7828998936237828)",0.856527,0.003998,"(0.8540483260302403, 0.8590049000279665)",0.365528,0.015665,...,1.142336,0.538762,"(0.8084076131365296, 1.4762634791990161)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
21,twitter_es_vanilla_bal_seed_,nyt,0.639087,0.088046,"(0.5845155413088085, 0.6936583717346699)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
23,twitter_es_vanilla_bal_seed_,ws,0.770015,0.025544,"(0.7541827073545508, 0.7858477340457535)",0.781363,0.007915,"(0.7764569630741037, 0.7862685829847756)",0.367121,0.020248,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
1,majority_gab_es_reg_nb0_h1_bal_seed_,nyt,0.917362,0.024158,"(0.9023890204927413, 0.9323356171884182)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN


In [37]:
task_to_learner_results.to_csv('../data/contextualize_results/task_to_learner.csv', index=False)
other_tasks_results.to_csv('../data/contextualize_results/other_tasks_with_learner.csv', index=False)
